In [ ]:
from google.colab import files
import ipywidgets as widgets
from keras.models import load_model
from google.colab import drive
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2, l1
from PIL import Image
import sys
import io
import imghdr

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

In [ ]:
base_dir = '/content/drive/MyDrive/tiff'

Функция загрузки предварительно обученных моделей

In [ ]:
def crate_model(path_weights):
  img_width, img_height = 122, 122
  input_shape = (img_width, img_height, 3)
  base_model_vgg = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
  for layer in base_model_vgg.layers:
    layer.trainable = False

  for layer in base_model_vgg.layers[15:]:
    layer.trainable = True
  model_vgg = Sequential()
  model_vgg.add(base_model_vgg)
  model_vgg.add(Flatten())
  model_vgg.add(Dense(1024, activation='relu'))
  model_vgg.add(BatchNormalization())
  model_vgg.add(Dropout(0.4))
  model_vgg.add(Dense(512, activation='relu'))
  model_vgg.add(BatchNormalization())
  model_vgg.add(Dropout(0.4))
  model_vgg.add(Dense(256, activation='relu'))
  model_vgg.add(BatchNormalization())
  model_vgg.add(Dropout(0.4))

  model_vgg.add(Dense(1, activation='sigmoid',
                      kernel_regularizer=tf.keras.regularizers.l2(0.01)))

  model_vgg.load_weights(path_weights)
  return model_vgg

In [ ]:
model_1 = crate_model('/content/drive/MyDrive/Снимки_tif/weights/model_vgg19_png_1_new_1206.weights.h5')
model_2 = crate_model('/content/drive/MyDrive/Снимки_tif/weights/model_vgg19_png_2_new_1206.weights.h5')
model_3 = crate_model('/content/drive/MyDrive/Снимки_tif/weights/model_vgg19_png_3_new_1206.weights.h5')
model_4 = crate_model('/content/drive/MyDrive/Снимки_tif/weights/model_vgg19_png_4_new_1206.weights.h5')

Функция обработки изображений

In [ ]:
def image_create(image_path):
    png_path = image_path.split('.')[0] + '.png'
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    normalized_image = cv2.normalize(image, None, 0, 65535, cv2.NORM_MINMAX)
    resized_image = cv2.resize(normalized_image, (122, 122), interpolation=cv2.INTER_CUBIC)
    resized_image_pil = Image.fromarray(resized_image)
    resized_image_pil.save(png_path, "PNG", compress_level=0, bits=16)
    return png_path


Функция предсказания вида агрокультур

In [ ]:
def predict_image(png_path, model_vgg):
    png_image = cv2.imread(str(png_path))
    input_image = np.expand_dims(png_image, axis=0)
    predictions = model_vgg.predict(input_image)
    pr_rounded = np.round(predictions, 3)
    return pr_rounded[0, 0]

Функция проверки формата загружаемых данных

In [ ]:
def check_file_type(filename, file_content):
    image_type = imghdr.what(None, h=file_content)
    if image_type:
        return True
    else:
        print(f'Файл {filename} не является изображением. Файл не будет загружен.')
        return False

Функция загрузки файлов

In [ ]:
uploaded_files = []

def upload_file(button):
    global uploaded_files
    uploaded_files = []
    upload = files.upload()

    for filename, file_content in upload.items():
        if check_file_type(filename, file_content):
            file_paths = [filename]
            uploaded_files.extend(file_paths)
            print('Файл(ы) загружен(ы):', ', '.join(file_paths))


Функция очистки данных

In [ ]:
def clear_data(button):
    uploaded_files.clear()
    clear_output()
    print('Данные удалены')

Функция выполнения последовательных предсказаний вероятности принадлежности каждого загруженного изображения к заданным классам

In [ ]:
def main(button):
    errors_count = 0
    culture_probabilities = {
        "пшеница яровая": model_1,
        "пшеница озимая": model_2,
        "свёкла сахарная": model_3,
        "лён": model_4
    }

    if not uploaded_files:
        print("Отсутствуют загруженные файлы. Программа завершает работу.")
        return

    for file_path in uploaded_files:
        png_path = image_create(file_path)

        results = {}
        for culture, model in culture_probabilities.items():
            result = predict_image(png_path, model)
            results[culture] = result

        print(f"На снимке {png_path} присутствует:")
        for culture, result in results.items():
            print(f"{culture} с вероятностью: {result:.3f}")

        print("_______________________________________________________________________________________________________________________________________________________\n")


Интерфейс

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import HBox, VBox

upload_button = widgets.Button(description='Загрузить файл')
upload_button.on_click(upload_file)

recognize_button = widgets.Button(description='Распознать')
recognize_button.on_click(main)


clear_button = widgets.Button(description='Очистить данные')
clear_button.on_click(clear_data)

buttons_hbox = HBox([upload_button, recognize_button, clear_button])
display(buttons_hbox)